using:
    https://github.com/aws/amazon-sagemaker-examples/blob/0efd885ef2a5c04929d10c5272681f4ca17dac17/advanced_functionality/custom-training-containers/basic-training-container/notebook/basic_training_container.ipynb
    and https://github.com/aws/amazon-sagemaker-examples/blob/0efd885ef2a5c04929d10c5272681f4ca17dac17/advanced_functionality/custom-training-containers/framework-container/notebook/framework-container.ipynb
    for permissions:  https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/
    consider also:  https://github.com/aws/amazon-sagemaker-examples/blob/0efd885ef2a5c04929d10c5272681f4ca17dac17/advanced_functionality/pipe_bring_your_own/pipe_bring_your_own.ipynb

In [2]:
!pip install sagemaker-studio-image-build

  Using cached sagemaker_studio_image_build-0.6.0-py3-none-any.whl


UsageError: Line magic function `%aws` not found.


In [3]:
%cd SAC_Docker/DockerSAC/sac-docker-app
!pwd

[Errno 2] No such file or directory: 'SAC_Docker/DockerSAC/sac-docker-app'
/root/SAC_Docker/DockerSAC/sac-docker-app
/root/SAC_Docker/DockerSAC/sac-docker-app


In [4]:
import pprint
import json

def print_json_object(json_object):
    pprint.pprint(json_object)


def load_json_object(json_file_path):
    with open(json_file_path) as json_file:
        return json.load(json_file)


In [5]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

from sagemaker.tuner import (
IntegerParameter,
CategoricalParameter,
ContinuousParameter,
HyperparameterTuner,
)

ecr_namespace = "sagemaker-training-containers/"
prefix = "sac-training-container"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(ecr_repository_name)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
419483997767
us-east-2
arn:aws:iam::419483997767:role/service-role/AmazonSageMaker-ExecutionRole-20240615T151259
sagemaker-training-containers/sac-training-container


In [7]:
#follow to get container in right place https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi-sdk-add-container-image.html
!sm-docker build -t ml-docker-app .

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
..................[Container] 2024/09/28 14:01:54.360682 Running on CodeBuild On-demand

[Container] 2024/09/28 14:01:54.360700 Waiting for agent ping
[Container] 2024/09/28 14:01:57.573102 Waiting for DOWNLOAD_SOURCE
[Container] 2024/09/28 14:01:57.838426 Phase is DOWNLOAD_SOURCE
[Container] 2024/09/28 14:01:57.878295 CODEBUILD_SRC_DIR=/codebuild/output/src4039554946/src
[Container] 2024/09/28 14:01:57.879722 YAML location is /codebuild/output/src4039554946/src/buildspec.yml
[Container] 2024/09/28 14:01:57.881575 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2024/09/28 14:01:57.881676 Processing environment variables
[Container] 2024/09/28 14:01:57.922329 No runtime version selected in buildspec.
[Container] 2024/09/28 14:01:57.937095 Moving to directory /codebuild/

In [14]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
print(container_image_uri)

419483997767.dkr.ecr.us-east-2.amazonaws.com/sagemaker-training-containers/sac-training-container:latest


In [8]:
#follow to get container in right place https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi-sdk-add-container-image.html
container_image_uri= "419483997767.dkr.ecr.us-east-2.amazonaws.com/sagemaker-studio-d-srdvznxntmpy:default-1718479071871"

In [59]:
#try local per https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-training-container.html

45

In [9]:
import tarfile
import os


def create_tar_file(source_files, target=None):
    if target:
        filename = target
    else:
        _, filename = tempfile.mkstemp()

    with tarfile.open(filename, mode="w:gz") as t:
        for sf in source_files:
            # Add all files from the directory into the root of the directory structure of the tar
            t.add(sf, arcname=os.path.basename(sf))
    return filename


create_tar_file(["sacapp.py"], "sourcedir.tar.gz")

'sourcedir.tar.gz'

In [10]:
sources = sagemaker_session.upload_data("sourcedir.tar.gz", bucket, prefix + "/code")
print(sources)
! rm sourcedir.tar.gz

s3://sagemaker-us-east-2-419483997767/sac-training-container/code/sourcedir.tar.gz


In [11]:
#test estimator briefly
import sagemaker
import json

# JSON encode hyperparameters.
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters(
    {
        "sagemaker_program": "sacapp.py",
        "sagemaker_submit_directory": sources,
        "n_layer": 1,
        "layer_size": 2,
        "alpha": 0.3,
        "maxT": 10000,
    }
)

est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    #instance_type="local",  # use local mode
    instance_type='ml.c4.xlarge',
    base_job_name=prefix,
    hyperparameters=hyperparameters,
)

est.fit()

INFO:sagemaker:Creating training-job with name: sac-training-container-2024-09-28-14-08-05-945


2024-09-28 14:08:06 Starting - Starting the training job...
2024-09-28 14:08:22 Starting - Preparing the instances for training...
2024-09-28 14:08:45 Downloading - Downloading input data...
2024-09-28 14:09:25 Training - Training image download completed. Training in progress..2024-09-28 14:09:32,038 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-09-28 14:09:32,039 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-09-28 14:09:32,050 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-09-28 14:09:32,050 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-09-28 14:09:32,060 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-09-28 14:09:32,060 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-09-28 14:09:32,070 sagemaker-training-toolkit INFO     I

In [12]:
#execute hyperparameter tuning job

objective_metric_name = "Reward"
objective_type = "Maximize"
metric_definitions = [
    {
        "Name": "Reward",
        "Regex": "Reward: (-[0-9\\.]+)",
    }
]


hyperparameter_ranges = {
"n_layer": CategoricalParameter([1,2,3,4,5,6,7,8]),
"layer_size": CategoricalParameter([3,4,5,6,7,8,9,10,11,12]),
"alpha": ContinuousParameter(0.01, 0.3),
}
#need to change hyperparmeters 

tuner = HyperparameterTuner(
est,
objective_metric_name,
hyperparameter_ranges,
metric_definitions,
max_jobs=21,
max_parallel_jobs=3,
objective_type=objective_type,
early_stopping_type="Auto",
strategy = "Bayesian",
base_tuning_job_name = 'SACrun1',
)

tuner.fit()

INFO:sagemaker:Creating hyperparameter tuning job with name: SACrun1-240928-1414


.........................................................................................................................................!
